In [1]:
import os
import time
import torch
import numpy as np
from utils import *
from h5py import File
import pennylane as qml
from typing import List
from qudit_mapping import *
from qutrit_synthesis import *
from scipy.io import loadmat, savemat

np.set_printoptions(precision=8, linewidth=200)
torch.set_printoptions(precision=8, linewidth=200)

In [ ]:
import time
import torch
import numpy as np
import pennylane as qml
from typing import List
from utils import updatemat
from qutrit_synthesis import NUM_PR, two_qutrit_unitary_synthesis

np.set_printoptions(precision=8, linewidth=200)
torch.set_printoptions(precision=8, linewidth=200)


def spin_operator(obj: List[int]):
    if len(obj) != 2:
        raise ValueError(f'The number of object qubits {len(obj)} should be 2')
    sx = qml.X(obj[0]) / 2 + qml.X(obj[1]) / 2
    sy = qml.Y(obj[0]) / 2 + qml.Y(obj[1]) / 2
    sz = qml.Z(obj[0]) / 2 + qml.Z(obj[1]) / 2
    return sx + sy + sz


def spin_operator2(obj: List[int]):
    if len(obj) != 2:
        raise ValueError(f'The number of object qubits {len(obj)} should be 2')
    s1 = qml.X(obj[0]) + qml.Y(obj[0]) + qml.Z(obj[0])
    s2 = qml.X(obj[1]) + qml.Y(obj[1]) + qml.Z(obj[1])
    return 3 / 2 * qml.I(obj) + (s1 @ s2) / 2


def Hamiltonian(n_qudits: int, beta: float):
    Ham = 0
    for i in range(n_qudits - 1):
        obj1 = [2 * i, 2 * i + 1]
        obj2 = [2 * i + 2, 2 * i + 3]
        Ham += spin_operator(obj1) @ spin_operator(obj2)
        Ham -= beta * (spin_operator2(obj1) @ spin_operator2(obj2))
    return Ham


def qutrit_symmetric_ansatz(n_qudits: int, params: torch.Tensor, Ham):
    params = params.reshape(n_qudits - 1, NUM_PR)
    n_qubits = 2 * n_qudits
    for i in range(n_qudits - 1):
        obj = list(range(n_qubits - 2 * i - 4, n_qubits - 2 * i))
        two_qutrit_unitary_synthesis(params[i], obj)
    return qml.expval(Ham)


n_qudits, beta, epochs, lr = 4, -0.3, 10, 0.1

n_qubits = 2 * n_qudits
dev = qml.device('default.qubit', n_qubits)
print(f'Coefficient beta: {beta}')
print(f'Number of qudits: {n_qudits}')
print(f'Number of qubits: {n_qubits}')

if torch.cuda.is_available() and n_qubits > 14:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'PyTorch Device: {device}')

pr_num = (n_qudits - 1) * NUM_PR
init_params = np.random.uniform(-np.pi, np.pi, pr_num)
params = torch.tensor(init_params, device=device, requires_grad=True)
cost_fn = qml.QNode(qutrit_symmetric_ansatz, dev, interface='torch')
optimizer = torch.optim.Adam([params], lr=lr)
Ham = Hamiltonian(n_qudits, beta)

start = time.perf_counter()
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = cost_fn(n_qudits, params, Ham)
    loss.backward()
    optimizer.step()
    count = epoch + 1
    if count % 10 == 0:
        t = time.perf_counter() - start
        print(f'Loss: {loss.item():.20f}, {count}/{epochs}, {t:.2f}')

loss_res = loss.detach().cpu()
params_res = optimizer.param_groups[0]['params'][0].detach().cpu()
time_str = time.strftime('%Y%m%d_%H%M%S', time.localtime())
mat_dict = {'n_qudits': n_qudits, 'n_qubits': n_qubits, 'beta': beta, 'epochs': epochs, \
'learning_rate': lr, 'params_init': init_params, 'params_res': params_res, 'loss': loss_res}
# updatemat(f'./mats/testVQE_{time_str}.mat', mat_dict)
print(mat_dict)

In [1]:
import time
import torch
import numpy as np
import pennylane as qml
from typing import List
from utils import updatemat
from qutrit_synthesis import NUM_PR, two_qutrit_unitary_synthesis


def spin_operator(obj: List[int]):
    if len(obj) != 2:
        raise ValueError(f'The number of object qubits {len(obj)} should be 2')
    sx = qml.X(obj[0]) / 2 + qml.X(obj[1]) / 2
    sy = qml.Y(obj[0]) / 2 + qml.Y(obj[1]) / 2
    sz = qml.Z(obj[0]) / 2 + qml.Z(obj[1]) / 2
    return sx + sy + sz


def spin_operator2(obj: List[int]):
    if len(obj) != 2:
        raise ValueError(f'The number of object qubits {len(obj)} should be 2')
    s1 = qml.X(obj[0]) + qml.Y(obj[0]) + qml.Z(obj[0])
    s2 = qml.X(obj[1]) + qml.Y(obj[1]) + qml.Z(obj[1])
    return 3 / 2 * qml.I(obj) + (s1 @ s2) / 2


def Hamiltonian(n_qudits: int, beta: float):
    Ham = 0
    for i in range(n_qudits - 1):
        obj1 = [2 * i, 2 * i + 1]
        obj2 = [2 * i + 2, 2 * i + 3]
        Ham += spin_operator(obj1) @ spin_operator(obj2)
        Ham -= beta * (spin_operator2(obj1) @ spin_operator2(obj2))
    return Ham


def qutrit_symmetric_ansatz(n_qudits: int, params: torch.Tensor, Ham):
    params = params.reshape(n_qudits - 1, NUM_PR)
    n_qubits = 2 * n_qudits
    for i in range(n_qudits - 1):
        obj = list(range(n_qubits - 2 * i - 4, n_qubits - 2 * i))
        two_qutrit_unitary_synthesis(params[i], obj)
    return qml.expval(Ham)


n_qudits = 2
n_qubits = 2 * n_qudits
dev = qml.device('default.qubit', n_qubits)
print(f'Number of qudits: {n_qudits}')
print(f'Number of qubits: {n_qubits}')

if torch.cuda.is_available() and n_qubits > 14:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'PyTorch Device: {device}')

np.random.seed(42)
pr_num = (n_qudits - 1) * NUM_PR
init_params = np.random.uniform(-np.pi, np.pi, pr_num)
params = torch.tensor(init_params, device=device, requires_grad=True)
cost_fn = qml.QNode(qutrit_symmetric_ansatz, dev, interface='torch')
optimizer = torch.optim.Adam([params], lr=0.1)
Ham = Hamiltonian(n_qudits, beta=-1 / 3)

epochs = 100
start = time.perf_counter()
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = cost_fn(n_qudits, params, Ham)
    loss.backward()
    optimizer.step()
    count = epoch + 1
    if count % 10 == 0:
        t = time.perf_counter() - start
        print(f'{count:3d}/{epochs}, Loss: {loss.item():.20f}, {t:.2f}')
params_res = optimizer.param_groups[0]['params'][0].detach()
print(params_res)

Number of qudits: 2
Number of qubits: 4
PyTorch Device: cpu
 10/100, Loss: 0.11452233784747231637, 1.92
 20/100, Loss: 0.04348994287393345010, 3.78
 30/100, Loss: 0.01553885829701937005, 5.67
 40/100, Loss: 0.00298989540027944783, 7.62
 50/100, Loss: 0.00208902553650319386, 9.45
 60/100, Loss: 0.00076402399449075273, 11.29
 70/100, Loss: 0.00005343525472996033, 13.24
 80/100, Loss: 0.00005210247622969288, 15.09
 90/100, Loss: 0.00004616149155771410, 16.96
100/100, Loss: 0.00001883485911411201, 18.88
tensor([-0.7883,  3.1787,  1.5731,  0.7353, -2.4910, -1.7185, -2.3456,  2.5369,
         0.1404,  1.3074, -3.0123,  2.9525,  2.5515, -2.1499, -2.6950, -2.5657,
        -0.8062,  0.4495, -0.0290, -1.0358,  1.0673, -2.2651, -1.3060, -0.8397,
        -0.4823,  1.7283, -2.2659,  0.4237,  0.2579, -2.5819,  1.0432, -1.5874,
        -2.9493,  2.8204,  2.9257,  2.0735, -1.3506, -2.2652,  0.9351, -0.1973,
        -2.5656, -0.2673, -3.3031,  2.8470, -1.5156,  1.0212, -1.1831, -0.1073,
         0.2107

In [2]:
def spin_operator(obj: List[int]):
    if len(obj) != 2:
        raise ValueError(f'The number of object qubits {len(obj)} should be 2')
    sx = qml.X(obj[0]) / 2 + qml.X(obj[1]) / 2
    sy = qml.Y(obj[0]) / 2 + qml.Y(obj[1]) / 2
    sz = qml.Z(obj[0]) / 2 + qml.Z(obj[1]) / 2
    return sx + sy + sz


def spin_operator2(obj: List[int]):
    if len(obj) != 2:
        raise ValueError(f'The number of object qubits {len(obj)} should be 2')
    s1 = qml.X(obj[0]) + qml.Y(obj[0]) + qml.Z(obj[0])
    s2 = qml.X(obj[1]) + qml.Y(obj[1]) + qml.Z(obj[1])
    return 3 / 2 * qml.I(obj) + (s1 @ s2) / 2


def Hamiltonian(n_qudits: int, beta: float):
    Ham = 0
    for i in range(n_qudits - 1):
        obj1 = [2 * i, 2 * i + 1]
        obj2 = [2 * i + 2, 2 * i + 3]
        Ham += spin_operator(obj1) @ spin_operator(obj2)
        Ham -= beta * (spin_operator2(obj1) @ spin_operator2(obj2))
        print(i, obj1, obj2)
    return Ham


n_qudits = 2
Ham = Hamiltonian(n_qudits, 1)
Ham_mat = Ham.matrix()
print(Ham_mat, Ham_mat.shape)

0 [0, 1] [2, 3]
[[-3. +0.j  -0.5+0.5j -0.5+0.5j  0. +0.j  -0.5+0.5j  0. +0.j   0. +2.j   0.5+0.5j -0.5+0.5j  0. +2.j   0. +0.j   0.5+0.5j  0. +0.j   0.5+0.5j  0.5+0.5j  1. +0.j ]
 [-0.5-0.5j -2. +0.j   0. +0.j  -0.5+0.5j  0. +0.j  -0.5+0.5j -0.5+0.5j  0. +1.j  -2. +0.j   1.5-1.5j -0.5+0.5j  0. -1.j  -0.5+0.5j  0. -1.j   0. +1.j  -0.5-0.5j]
 [-0.5-0.5j  0. +0.j  -2. +0.j  -0.5+0.5j -2. +0.j  -0.5+0.5j  1.5-1.5j  0. -1.j   0. +0.j  -0.5+0.5j -0.5+0.5j  0. +1.j  -0.5+0.5j  0. +1.j   0. -1.j  -0.5-0.5j]
 [ 0. +0.j  -0.5-0.5j -0.5-0.5j -1. +0.j  -0.5-0.5j -1. +0.j   1. +0.j   0.5-0.5j -0.5-0.5j  1. +0.j  -1. +0.j   0.5-0.5j -1. +0.j   0.5-0.5j  0.5-0.5j  0. +0.j ]
 [-0.5-0.5j  0. +0.j  -2. +0.j  -0.5+0.5j -2. +0.j  -0.5+0.5j  1.5-1.5j  0. -1.j   0. +0.j  -0.5+0.5j -0.5+0.5j  0. +1.j  -0.5+0.5j  0. +1.j   0. -1.j  -0.5-0.5j]
 [ 0. +0.j  -0.5-0.5j -0.5-0.5j -1. +0.j  -0.5-0.5j -1. +0.j   1. +0.j   0.5-0.5j -0.5-0.5j  1. +0.j  -1. +0.j   0.5-0.5j -1. +0.j   0.5-0.5j  0.5-0.5j  0. +0.j ]
 [ 0. 

In [2]:
Sx = {2: np.array([[0, 1], [1, 0]], dtype=CDTYPE) / 2,  \
      3: np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]], dtype=CDTYPE) / np.sqrt(2)}
Sy = {2: np.array([[0, -1j], [1j, 0]], dtype=CDTYPE) / 2, \
      3: np.array([[0, -1j, 0], [1j, 0, -1j], [0, 1j, 0]], dtype=CDTYPE) / np.sqrt(2)}
Sz = {2: np.array([[1, 0], [0, -1]], dtype=CDTYPE) / 2, \
      3: np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]], dtype=CDTYPE)}

dim = 3
print(Sx[dim])
print(Sy[dim])
print(Sz[dim])

S = Sx[dim] + Sy[dim] + Sz[dim]
print(S)

[[0.        +0.j 0.70710678+0.j 0.        +0.j]
 [0.70710678+0.j 0.        +0.j 0.70710678+0.j]
 [0.        +0.j 0.70710678+0.j 0.        +0.j]]
[[ 0.+0.j         -0.-0.70710678j  0.+0.j        ]
 [ 0.+0.70710678j  0.+0.j         -0.-0.70710678j]
 [ 0.+0.j          0.+0.70710678j  0.+0.j        ]]
[[ 1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.+0.j]]
[[ 1.        +0.j          0.70710678-0.70710678j  0.        +0.j        ]
 [ 0.70710678+0.70710678j  0.        +0.j          0.70710678-0.70710678j]
 [ 0.        +0.j          0.70710678+0.70710678j -1.        +0.j        ]]


In [ ]:
import time
import torch
import numpy as np
from h5py import File
import pennylane as qml
from qudit_mapping import symmetric_encoding
from qutrit_synthesis import NUM_PR, two_qutrit_unitary_synthesis

path = '../QuditVQE/data_232/from_classical_to_violation_dense'
s = File(f'{path}/232_d3_D9_model1216_RDM2_iter1_target_state_vector.mat')
state = s['target_state_vec'][:].view('complex')
n_qudits = s['N'][0]
s.close()

dim = 3
n_qubits = n_qudits * (dim - 1)
dev = qml.device('default.qubit', n_qubits)
print(f'Number of qudits: {n_qudits}')
print(f'Number of qubits: {n_qubits}')

if torch.cuda.is_available() and n_qubits > 14:
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f'PyTorch Device: {device}')

psi_sym = symmetric_encoding(state, n_qudits)
psi_sym = torch.from_numpy(psi_sym)

pr_num = (n_qudits - 1) * NUM_PR
init_params = np.random.uniform(-np.pi, np.pi, pr_num)
params = torch.tensor(init_params, device=device, requires_grad=True)
optimizer = torch.optim.Adam([params], lr=0.1)


@qml.qnode(dev, interface='torch')
def qutrit_symmetric_ansatz(params: torch.Tensor):
    params = params.reshape(n_qudits - 1, NUM_PR)
    for i in range(n_qudits - 1):
        obj = list(range(n_qubits - (dim - 1) * (i + 2), n_qubits - (dim - 1) * i))
        two_qutrit_unitary_synthesis(params[i], obj)
    return qml.state()


def cost_fn(params: torch.Tensor):
    psi_circ = qutrit_symmetric_ansatz(params)
    fidelity = torch.abs(psi_circ.conj() @ psi_sym)**2
    return fidelity


epochs = 500
start = time.perf_counter()
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = cost_fn(params)
    loss.backward()
    optimizer.step()
    count = epoch + 1
    if count % 10 == 0:
        t = time.perf_counter() - start
        print(f'{count:3d}/{epochs}, Loss: {loss.item():.15f}, Fidelity: {1-loss.item():.15f}, {t:.2f}')